<a href="https://colab.research.google.com/github/aabdulhakeem/Forward-and-backward-chaining/blob/main/forwardandbackward.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
facts = {}
rules = {}

In [2]:
def convert_fact(line):
    fact = line.strip().split()
    if len(fact) > 1:
      if fact[1] == '>=' or fact[1] == '<=' or fact[1] == '<' or fact[1] == '>' or fact[1] == '=':
        return fact[0], (fact[1], int(fact[2]))
      else:
        return fact[0], fact[-1]

    elif len(fact) == 1:
        return fact[0], True
    else:
        return None, None

In [3]:
def add_fact(line, facts):
    var, val = convert_fact(line)
    facts[var] = val

In [4]:
with open('content/facts.txt', 'r') as fact_file:
    for line in fact_file:
        add_fact(line, facts)

In [5]:
facts
# test = 'seeds = 0'
# test2 ='skin_smell'
# ans = test.strip().split()
# ans2 = test2.strip().split()

# print(len(ans))  # 3
# print(len(ans2)) # 1

{'seeds': ('=', 0),
 'diameter': ('=', 7),
 'skin_smell': True,
 'color': 'orange',
 '#goal': 'citrus_fruit'}

In [6]:
def convert_rule(line):
    rule = line.strip().split(' THEN ')
    if_part = rule[0].strip()[3:]
    then_part = rule[1].strip()
    return if_part, then_part

In [7]:
def add_rule(line, i):
    if_part, then_part = convert_rule(line)
    rules[(if_part, i)] = then_part   # if this, then that

In [8]:
with open('content/rules.txt', 'r') as rule_file:
    i = 1
    for line in rule_file:
        add_rule(line, i)
        i += 1

In [9]:
rules

{('shape is long AND color is yellow', 1): 'fruit is banana',
 ('shape is round AND color is red AND size is medium', 2): 'fruit is apple',
 ('shape is round AND color is red AND size is small', 3): 'fruit is cherry',
 ('skin_smell', 4): 'perfumed',
 ('fruit is lemon OR fruit is orange OR fruit is pomelo OR fruit is grapefruit',
  5): 'citrus_fruit',
 ('size is medium AND color is yellow AND perfumed', 6): 'fruit is lemon',
 ('size is medium AND color is green', 7): 'fruit is kiwi',
 ('size is big AND perfumed AND color is orange AND citrus_fruit',
  8): 'fruit is grapefruit',
 ('perfumed AND color is orange AND size is medium', 9): 'fruit is orange',
 ('perfumed AND color is red AND size is small AND seeds = 0',
  10): 'fruit is strawberry',
 ('diameter < 2', 11): 'size is small',
 ('diameter > 10', 12): 'size is big',
 ('diameter > 2 AND diameter < 10', 13): 'size is medium'}

In [10]:
def compare(a, op, b, op2):
  if op2 == '=':
    if op == '>=':
        return a >= b
    elif op == '<=':
        return a <= b
    elif op == '<':
        return a < b
    elif op == '>':
        return a > b
    elif op == '=':
        return a == b

In [11]:
def check_rule(if_part, facts):
    check_list = []
    And = False
    Or = False
    if 'AND' in if_part:
        And = True
        if_part = if_part.split(' AND ')
    elif 'OR' in if_part:
        Or = True
        if_part = if_part.split(' OR ')
    else:
        if_part = [if_part]

    for part in if_part:
        var, val = convert_fact(part)
        if isinstance(val, tuple):
            comp = compare(facts.get(var)[1], val[0], val[1], facts.get(var)[0])  # compare(a, op, b, op2)
            check_list.append(comp)
        else:
            if facts.get(var) == val:
                check_list.append(True)
            else:
                check_list.append(False)

    if And:
        if all(check_list):
            return True
        else:
            return False
    elif Or:
        if any(check_list):
            return True
        else:
            return False
    else:
        return check_list[0]

In [12]:
goal = 'citrus_fruit'
value = True

In [13]:
def forward_chaining(goal, value):
    cycle = 0
    rules_copy = rules.copy()
    facts_copy = facts.copy()
    print(f"before starting, initial facts: {facts_copy}\n")
    while rules_copy:
        cycle += 1
        fire = False
        for if_part in rules_copy:
            if check_rule(if_part[0], facts_copy): 
                fire = True
                add_fact(rules_copy[if_part], facts_copy)
                i = if_part[1]
                print(f"cycle {cycle}, R{i}:({if_part[0]} -> {rules_copy[if_part]}) fired")
                print(f"facts: {facts_copy}\n")
                del rules_copy[if_part]
                break
        if facts_copy.get(goal) == value:
            print(f"cycle {cycle}, goal reached, facts: {facts_copy}")
            return True
            break
        elif not fire:
            print(f"cycle {cycle}, no rule fired, facts: {facts_copy}")
            return False
            break

In [14]:
forward_chaining(goal, value)

before starting, initial facts: {'seeds': ('=', 0), 'diameter': ('=', 7), 'skin_smell': True, 'color': 'orange', '#goal': 'citrus_fruit'}

cycle 1, R4:(skin_smell -> perfumed) fired
facts: {'seeds': ('=', 0), 'diameter': ('=', 7), 'skin_smell': True, 'color': 'orange', '#goal': 'citrus_fruit', 'perfumed': True}

cycle 2, R13:(diameter > 2 AND diameter < 10 -> size is medium) fired
facts: {'seeds': ('=', 0), 'diameter': ('=', 7), 'skin_smell': True, 'color': 'orange', '#goal': 'citrus_fruit', 'perfumed': True, 'size': 'medium'}

cycle 3, R9:(perfumed AND color is orange AND size is medium -> fruit is orange) fired
facts: {'seeds': ('=', 0), 'diameter': ('=', 7), 'skin_smell': True, 'color': 'orange', '#goal': 'citrus_fruit', 'perfumed': True, 'size': 'medium', 'fruit': 'orange'}

cycle 4, R5:(fruit is lemon OR fruit is orange OR fruit is pomelo OR fruit is grapefruit -> citrus_fruit) fired
facts: {'seeds': ('=', 0), 'diameter': ('=', 7), 'skin_smell': True, 'color': 'orange', '#goal': '

True

In [15]:
def backward_chaining(goal, value, facts, rules, used_rules=None, deduced_facts=None):
    if used_rules is None:
        used_rules = set()
    if deduced_facts is None:
        deduced_facts = set()
    
    
    print(f"Goal: {goal} = {value}")
    
    # Check if already goal is given in facts 
    if goal in facts: # 
        fact_value = facts[goal] 
        if isinstance(fact_value, tuple): # if it is a tuple, check the operator and value
            op, val = fact_value
            if op == '=' and val == value:
                print(f"Already known: {goal} = {value}")
                return True
        elif fact_value == value: # if it is not a tuple, check the value directly
            print(f"Already known: {goal} = {value}")
            return True
    # or deduced it
    if (goal, value) in deduced_facts:
        print(f"Already deduced: {goal} = {value}")
        return True
    
    
    
    
    
    # Find applicable rules that conclude the goal to start with it 
    applicable_rules = []
    for rule_conditions, rule_conclusion in rules.items():
        
        conditions_str, rule_id = rule_conditions # Extract the conditions and rule ID
        concl_var, concl_val = convert_fact(rule_conclusion)
        if concl_var == goal and concl_val == value and rule_id not in used_rules:
            applicable_rules.append((conditions_str, rule_id))
    
    if not applicable_rules:
        print(f"No rules can conclude {goal} = {value}")
        return False
    
    
    # print (f"Applicable rules for {goal} = {value}: {applicable_rules}")    
    
    
    
    
    
    
    for conditions, rule_id in applicable_rules:
        print(f"Trying rule {rule_id}: IF {conditions} THEN {goal} = {value}")
        
        current_used_rules = used_rules.copy()
        if rule_id is not None:
            current_used_rules.add(rule_id)
        
        # Use check_rule to verify conditions
        if check_rule(conditions, facts):  # 
            facts[goal] = value
            deduced_facts.add((goal, value))
            print(f"Proved {goal} = {value} using rule {rule_id}")
            print(f"Updated facts: {facts}")
            return True
        else:
            # If check_rule fails, try to prove subgoals recursively
            if ' AND ' in conditions or ' OR ' in conditions:
                # Handle AND/OR conditions
                if ' AND ' in conditions:
                    condition_parts = conditions.split(' AND ')
                    is_and = True
                else:  # OR case
                    condition_parts = conditions.split(' OR ')
                    is_and = False
                
                proved = False
                current_deduced = set()
                
                for condition in condition_parts:
                    var, val = convert_fact(condition)
                    
                    # Recursive proof for each subgoal
                    if backward_chaining(var, val, facts, rules, current_used_rules, deduced_facts):
                        current_deduced.add((var, val))
                        if not is_and:  # OR condition met
                            proved = True
                            break
                    elif is_and:  # AND condition failed
                        break
                else:
                    # All conditions met for AND case
                    if is_and:
                        proved = True
                
                if proved:
                    facts[goal] = value
                    deduced_facts.update(current_deduced)
                    deduced_facts.add((goal, value))
                    print(f"Proved {goal} = {value} using rule {rule_id}")
                    print(f"Updated facts: {facts}")
                    return True
    
    print(f"Failed to prove {goal} = {value}")
    return False

In [16]:
goal = facts['#goal']
result = backward_chaining(goal, True, facts, rules)

Goal: citrus_fruit = True
Trying rule 5: IF fruit is lemon OR fruit is orange OR fruit is pomelo OR fruit is grapefruit THEN citrus_fruit = True
Goal: fruit = lemon
Trying rule 6: IF size is medium AND color is yellow AND perfumed THEN fruit = lemon
Goal: size = medium
Trying rule 13: IF diameter > 2 AND diameter < 10 THEN size = medium
Proved size = medium using rule 13
Updated facts: {'seeds': ('=', 0), 'diameter': ('=', 7), 'skin_smell': True, 'color': 'orange', '#goal': 'citrus_fruit', 'size': 'medium'}
Goal: color = yellow
No rules can conclude color = yellow
Failed to prove fruit = lemon
Goal: fruit = orange
Trying rule 9: IF perfumed AND color is orange AND size is medium THEN fruit = orange
Goal: perfumed = True
Trying rule 4: IF skin_smell THEN perfumed = True
Proved perfumed = True using rule 4
Updated facts: {'seeds': ('=', 0), 'diameter': ('=', 7), 'skin_smell': True, 'color': 'orange', '#goal': 'citrus_fruit', 'size': 'medium', 'perfumed': True}
Goal: color = orange
Alread